In [1]:
# Calcula la precipitación en días
# con temperatura media mayor a la
# media climatológica.

import os

import xarray as xr

In [5]:
path = "../../../resultados/" 

CHIRPS = "../../../datos/CHIRPS/"
CHIRTS =  "../../../resultados/CHIRTS/"

RCP = ["RCP4p5", "RCP8p5"]
fut = [2030, 2050, 2070]
years = list(range(2010, 2017))

dirs = ["Actual"] + RCP
path_d = [path + "vars/" + x + "/" for x in dirs]

# Variables de CHIRTS/CHIRPS.
vars = ["Tmax", "Tmin", "Tmean",
    "Pre", "Pre_Tmean"]

# Si no existen las carpetas, las crea.
for p in path_d:
    if not os.path.exists(p):
        os.mkdir(p)
    if not os.path.exists(p + vars[4]):
        os.mkdir(p + vars[4])

In [7]:
# Calculamos Pre_Tmean actual.

k = 0

ds_T = xr.open_dataset(path_d[k]
    + "mexico_Tmean_yearly.nc"
    ).rename(Tmean = "Pre")

for j in years:
    # Abrimos la temperatura media anual.
    with xr.open_dataset( CHIRTS +
        vars[2] + "/mexico_" + vars[2]
        + "." + str(j) + ".nc"
        ).rename(Tmean = "Pre")  as ds_mean:
        # Abrimos la precipitación.
        with xr.open_dataset( CHIRPS + "/mexico_" +
            vars[3] + "." + str(j) + ".nc" ) as ds_pre:
            # Establecemos np.nan como valor sin datos.
            ds_pre = ds_pre.where(ds_pre > -9000)
            # Quitamos los puntos donde la temperatura
            # es menor a la media climatológica.
            ds_pre = ds_pre.where( ds_mean > ds_T )
    
        # Guardamos el archivo.
        ds_pre = ds_pre.rename(Pre = vars[4])
        ds_pre.mean("time").to_netcdf( path_d[k]
            + vars[4] + "/mexico_" 
            + vars[4] + "." +
            str(j) + "_yearly.nc")

In [12]:
# Calculamos Pre_Tmean futuro.

# Repetimos el cálculo para cada RCP.
for k in range(1, 3):
    # Repetimos el cálculo para cada año.
    for f in fut:
        ds_T = xr.open_dataset( path_d[k] +
            vars[2] + "/mexico_Tmean." +
            str(f) + "_yearly.nc"
            ).rename(Tmean = "Pre")

        # Abrimos la temperatura media.
        with xr.open_dataset( path_d[k] +
            vars[2] + "/mexico_" + vars[2]
            + "." + str(f) + "_daily.nc"
            ).rename(Tmean = "Pre") as ds_mean:
            # Abrimos la precipitación.
            with xr.open_dataset( path_d[k] +
                vars[3] + "/mexico_" + vars[3] + "."
                + str(f) + "_daily.nc" ) as ds_pre:
                # Establecemos np.nan como valor sin datos.
                ds_pre = ds_pre.where(ds_pre > -9000)
                # Quitamos los puntos donde la temperatura
                # es menor a la media climatológica.
                ds_pre = ds_pre.where( ds_mean > ds_T )
    
            # Guardamos el archivo.
            ds_pre = ds_pre.rename(Pre = vars[4])
            ds_pre.mean("time").to_netcdf( path_d[k]
                + vars[4] + "/mexico_" + vars[4]
                + "." + str(f) + "_yearly.nc" )